In [1]:
import torch
import os
from omegaconf import OmegaConf, open_dict
from pytorch_lightning import Trainer

import nemo.collections.asr as nemo_asr
from pathlib import Path


In [2]:
# Load the model from a local .nemo file

MODEL_ROOT = "/external2/models/hf/stt_en_conformer_ctc_small/"
MODEL_ROOT = Path(MODEL_ROOT)
model_path = "/external2/models/hf/stt_en_conformer_ctc_small/stt_en_conformer_ctc_small.nemo"
model_path = MODEL_ROOT / "stt_en_conformer_ctc_small.nemo"

cfg = nemo_asr.models.ASRModel.restore_from(restore_path=model_path, return_config=True)

# If you need to get the configuration, you can access it from the loaded model

print(cfg)

{'sample_rate': 16000, 'log_prediction': True, 'ctc_reduction': 'mean_batch', 'train_ds': {'manifest_filepath': '/data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json', 'sample_rate': 16000, 'batch_size': 64, 'shuffle': True, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False, 'trim_silence': False, 'max_duration': 20.0, 'min_duration': 0.1, 'shuffle_n': 2048, 'is_tarred': True, 'tarred_audio_filepaths': '/data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar'}, 'validation_ds': {'manifest_filepath': ['/data/ASR/LibriSpeech/librispeech_withsp2/manifests/librivox-dev-other.json', '/data/ASR/LibriSpeech/librispeech_withsp2/manifests/librivox-dev-clean.json', '/data/ASR/LibriSpeech/librispeech_withsp2/manifests/librivox-test-other.json', '/data/ASR/LibriSpeech/librispeech_withsp2/manifests/librivox-test-clean.json'], 'sample_rate': 16000, 'batch_size': 64, 'shuffle': False, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False, 'is_tarred':

In [3]:
from nemo.core import adapter_mixins

# Utility method to check and update the model config
def update_model_config_to_support_adapter(model_cfg):
    with open_dict(model_cfg):
        adapter_metadata = adapter_mixins.get_registered_adapter(model_cfg.encoder._target_)
        if adapter_metadata is not None:
            model_cfg.encoder._target_ = adapter_metadata.adapter_class_path

    print("Updated encoder _target_ model :", model_cfg.encoder._target_)
    return model_cfg

In [4]:
cfg = update_model_config_to_support_adapter(cfg)

Updated encoder _target_ model : nemo.collections.asr.modules.conformer_encoder.ConformerEncoderAdapter


In [5]:
model = nemo_asr.models.ASRModel.restore_from(model_path, override_config_path=cfg)

[NeMo I 2024-07-14 01:07:48 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-07-14 01:07:48 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2024-07-14 01:07:48 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2024-07-14 01:07:48 features:289] PADDING: 0
[NeMo I 2024-07-14 01:07:49 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /external2/models/hf/stt_en_conformer_ctc_small/stt_en_conformer_ctc_small.nemo.


In [6]:
data_dir = "/home/ksingla/workspace/medical-ner/data/"
TRAIN_MANIFEST = os.path.join(data_dir, "text_train_cleaned_manifest.json")
TEST_MANIFEST = os.path.join(data_dir, "text_valid_cleaned_manifest.json")
ALL_TAGS = os.path.join(data_dir, "alltags.txt")
taglist = []
with open(ALL_TAGS, 'r') as f:
    for line in f:
        word, tag = line.split()
        taglist.append(tag)

In [7]:
import subprocess
import os
import sentencepiece as spm
import logging
import sys
import json

def generate_sentencepiece_model_pb2(script_dir, proto_file_path):
    # Construct the command
    command = [
        'protoc',
        f'--python_out={script_dir}',
        proto_file_path
    ]
    
    try:
        # Run the command
        subprocess.run(command, check=True)
        print("Successfully generated sentencepiece_model_pb2.py")
    except subprocess.CalledProcessError as e:
        print(f"Error generating sentencepiece_model_pb2.py: {e}")
        sys.exit(1)

def edit_spt_model(input_file, output_folder, tokens, vocab_file, vocab_txt_file, is_userdefined=False):
    from sentencepiece_model_pb2 import ModelProto  # Ensure this import is after the proto generation
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    output_model_file = os.path.join(output_folder, 'tokenizer.model')
    output_vocab_file = os.path.join(output_folder, 'tokenizer.vocab')
    output_vocab_txt_file = os.path.join(output_folder, 'vocab.txt')

    token_type = 3
    if is_userdefined:
        token_type = 4

    model = ModelProto()
    model.ParseFromString(open(input_file, 'rb').read())

    existing_tokens = {piece.piece for piece in model.pieces}

    new_tokens = []
    for token in tokens:
        if token in existing_tokens:
            logging.warning(f"Special Token '{token}' already exists in the input model, skipping.")
            continue
        piece = model.SentencePiece(piece=token, score=0.0, type=token_type)
        model.pieces.append(piece)
        new_tokens.append(token)

    sp = spm.SentencePieceProcessor()
    try:
        sp.LoadFromSerializedProto(model.SerializeToString())
        for token in new_tokens:
            id = sp.piece_to_id(token)
            logging.info(f"Created token '{token}' at ID {id}")
        logging.info(f"New tokenizer vocab size: {sp.get_piece_size()}")
    except:
        logging.error("Could not appropriately configure new tokenizer. Verify if the special tokens already exist.")
        sys.exit(1)

    with open(output_model_file, 'wb') as outf:
        outf.write(model.SerializeToString())

    logging.info(f"Created new tokenizer at: {output_model_file}")

    # Read the original vocab file and append the new tokens
    with open(vocab_file, 'r') as original_vocab_file:
        original_vocab = original_vocab_file.readlines()

    with open(output_vocab_file, 'w') as updated_vocab_file:
        updated_vocab_file.writelines(original_vocab)
        for token in new_tokens:
            updated_vocab_file.write(f"{token}\n")

    # Update vocab.txt
    with open(vocab_txt_file, 'r') as original_vocab_txt_file:
        original_vocab_txt = original_vocab_txt_file.readlines()

    with open(output_vocab_txt_file, 'w') as updated_vocab_txt_file:
        updated_vocab_txt_file.writelines(original_vocab_txt)
        for token in new_tokens:
            updated_vocab_txt_file.write(f"{token}\n")

    logging.info(f"Updated vocab files: {output_vocab_file}, {output_vocab_txt_file}")

def update_model_config(model, new_model_path):
    model['cfg']['tokenizer']['model_path'] = new_model_path
    logging.info(f"Updated model configuration with new tokenizer model path: {new_model_path}")


# Define input and output paths
input_folder = MODEL_ROOT / "tokenizer"
output_folder = MODEL_ROOT / "new_tokenizer"


#input_folder = '/external/ksingla/models/nemo/stt_en_conformer_ctc_small/tokenizer'
#output_folder = '/external/ksingla/models/nemo/stt_en_conformer_ctc_small/new_tokenizer'
#proto_dir = '/path/to/save/proto'  # Define the actual path where the proto file should be saved
#proto_file = '/path/to/sentencepiece_model.proto'  # Define the actual path to the sentencepiece_model.proto file

input_file = input_folder / 'tokenizer.model'
vocab_file = input_folder / 'tokenizer.vocab'
vocab_txt_file = input_folder / 'vocab.txt'

# input_file = os.path.join(input_folder, 'tokenizer.model')
# vocab_file = os.path.join(input_folder, 'tokenizer.vocab')
# vocab_txt_file = os.path.join(input_folder, 'vocab.txt')

# Include all single-digit integers in the tokens list
punctuations = ['.', ',', '?', '!', ';', ':', '-', '(', ')', '[', ']', '{', '}', '<', '>', '/', '\\', '|', '@', '#', '$', '%', '^', '&', '*', '+', '=', '~', '`', '_', '"', "'"]
tokens = taglist + [str(i) for i in range(10)] + punctuations
is_userdefined = True

# Step 1: Generate the sentencepiece_model_pb2.py file
#generate_sentencepiece_model_pb2(proto_dir, proto_file)

# Step 2: Edit the SentencePiece model
edit_spt_model(input_file, output_folder, tokens, vocab_file, vocab_txt_file, is_userdefined)

# Step 3: Load the model configuration and update it
# model_config_file = '/path/to/model/config.json'  # Define the actual path to the model config file
# with open(model_config_file, 'r') as f:
#     model = json.load(f)

# new_model_path = os.path.join(output_folder, 'tokenizer.model')
# update_model_config(model, new_model_path)

# # Save the updated model configuration
# with open(model_config_file, 'w') as f:
#     json.dump(model, f, indent=4)

#logging.info(f"Updated model configuration saved to: {model_config_file}")


In [8]:
model.change_vocabulary(output_folder, "bpe")

[NeMo W 2024-07-14 01:07:49 modelPT:258] You tried to register an artifact under config key=tokenizer.model_path but an artifact for it has already been registered.
[NeMo W 2024-07-14 01:07:49 modelPT:258] You tried to register an artifact under config key=tokenizer.vocab_path but an artifact for it has already been registered.
[NeMo W 2024-07-14 01:07:49 modelPT:258] You tried to register an artifact under config key=tokenizer.spe_tokenizer_vocab but an artifact for it has already been registered.


[NeMo I 2024-07-14 01:07:49 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1986 tokens
[NeMo I 2024-07-14 01:07:49 ctc_bpe_models:248] 
    Replacing old number of classes (1024) with new number of classes - 1986
[NeMo I 2024-07-14 01:07:50 ctc_bpe_models:290] Changed tokenizer to ['<unk>', 's', '▁the', '▁a', 't', '▁to', '▁and', '▁i', '▁of', "'", 'ed', '▁in', 'd', 'ing', 'n', 'e', '▁it', '▁that', '▁you', 'y', 'er', 'r', '▁for', 'm', '▁is', '▁he', 're', '▁was', '▁be', 'p', 'ly', '▁so', '▁we', 'a', 'g', 'o', '▁c', 'b', 'u', '▁on', '▁have', '▁but', 'll', '▁with', '▁re', 'or', '▁s', 'al', '▁do', '▁know', 'ar', '▁they', '▁not', '▁as', '▁this', 'in', 'le', '▁e', '▁are', '▁like', 'c', '▁uh', 'ri', '▁me', '▁his', '▁at', 'l', 'es', '▁de', '▁yeah', '▁can', 'k', '▁or', '▁my', '▁all', '▁had', '▁there', '▁will', '▁one', 'il', '▁no', '▁what', 'en', 'ck', '▁b', '▁f', 'ce', 'ch', 'i', '▁by', '▁she', '▁from', '▁an', 'ic', 'ur', 've', 'w', 'ter', 'la', '▁if', '▁just', 'th', 'li', '▁', '▁h

In [9]:
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
max_steps = 30000

trainer = Trainer(devices=1, accelerator=accelerator, max_steps=max_steps,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=50, check_val_every_n_epoch=1)

model.set_trainer(trainer)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
# utility method
import json
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest

In [11]:
with open_dict(model.cfg):
  # Train Dataloader
  model.cfg.train_ds.manifest_filepath = TRAIN_MANIFEST
  model.cfg.train_ds.batch_size = 32
  model.cfg.train_ds.is_tarred = False
  model.cfg.train_ds.tarred_audio_filepaths = None

  model.cfg.validation_ds.manifest_filepath = TEST_MANIFEST
  model.cfg.validation_ds.batch_size = 32

model.setup_training_data(model.cfg.train_ds)
model.setup_multiple_validation_data(model.cfg.validation_ds)
model.setup_multiple_test_data(model.cfg.validation_ds)

[NeMo I 2024-07-14 01:07:51 collections:196] Dataset loaded with 18682 files totalling 17.48 hours
[NeMo I 2024-07-14 01:07:51 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-07-14 01:07:51 collections:196] Dataset loaded with 2390 files totalling 2.36 hours
[NeMo I 2024-07-14 01:07:51 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-07-14 01:07:52 collections:196] Dataset loaded with 2390 files totalling 2.36 hours
[NeMo I 2024-07-14 01:07:52 collections:197] 0 files were filtered totalling 0.00 hours


In [12]:
with open_dict(model.cfg):
  # Spec Augment
  model.cfg.spec_augment.freq_masks = model.cfg.spec_augment.freq_masks  # Can be changed
  model.cfg.spec_augment.freq_width = model.cfg.spec_augment.freq_width  # Can be changed
  model.cfg.spec_augment.time_masks = model.cfg.spec_augment.time_masks  # Can be changed
  model.cfg.spec_augment.time_width = model.cfg.spec_augment.time_width  # Can be changed

model.spec_augmentation = model.from_config_dict(model.cfg.spec_augment)

In [13]:
if 'optim' in model.cfg:
  print(OmegaConf.to_yaml(model.cfg.optim))

name: adamw
lr: 2.0
betas:
- 0.9
- 0.98
weight_decay: 0
sched:
  name: NoamAnnealing
  d_model: 176
  warmup_steps: 10000
  warmup_ratio: null
  min_lr: 1.0e-06



In [14]:
with open_dict(model.cfg):
  model.cfg.optim.lr = 0.1
  model.cfg.optim.weight_decay = 0.0
  model.cfg.optim.sched.warmup_steps = 100

model.setup_optimization(model.cfg.optim);

[NeMo I 2024-07-14 01:07:52 modelPT:723] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.1
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2024-07-14 01:07:52 lr_scheduler:915] Scheduler "<nemo.core.optim.lr_scheduler.NoamAnnealing object at 0x778d318660e0>" 
    will be used during training (effective maximum steps = 30000) - 
    Parameters : 
    (d_model: 176
    warmup_steps: 100
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 30000
    )


In [15]:
if hasattr(model, 'adapter_module_names'):
  print(model.adapter_module_names)

['', 'encoder', 'decoder', 'joint']


In [16]:
for module in model.children():
  if hasattr(module, 'get_accepted_adapter_types'):
    types = module.get_accepted_adapter_types()
    print("Module : ", module.__class__.__name__)

    for tp in types:
      print(tp)
    print()

Module :  ConformerEncoderAdapter
<class 'nemo.collections.asr.parts.submodules.adapters.multi_head_attention_adapter_module.RelPositionMultiHeadAttentionAdapter'>
<class 'nemo.collections.asr.parts.submodules.adapters.multi_head_attention_adapter_module.MultiHeadAttentionAdapter'>
<class 'nemo.collections.common.parts.adapter_modules.LinearAdapter'>

Module :  ConvASRDecoder
<class 'nemo.collections.common.parts.adapter_modules.LinearAdapter'>



In [17]:
from nemo.collections.common.parts.adapter_modules import LinearAdapterConfig

In [18]:
#%% [code]
#@title Adapter Setup { display-mode: "form" }
adapter_name = "AN4" #@param {type:"string"}
adapter_dim = 32 #@param {type:"integer"}
adapter_activation = "swish" #@param {type:"string"}
adapter_norm_position = "pre" #@param ["pre", "post"]

In [19]:
adapter_cfg = LinearAdapterConfig(
    in_features=model.cfg.encoder.d_model,  # conformer specific model dim. Every layer emits this dim at its output.
    dim=adapter_dim,  # the bottleneck dimension of the adapter
    activation=adapter_activation,  # activation used in bottleneck block
    norm_position=adapter_norm_position,  # whether to use LayerNorm at the beginning or the end of the adapter
)
print(adapter_cfg)

LinearAdapterConfig(in_features=176, dim=32, activation='swish', norm_position='pre', dropout=0.0, adapter_strategy=ResidualAddAdapterStrategyConfig(stochastic_depth=0.0, l2_lambda=0.0, _target_='nemo.core.classes.mixins.adapter_mixin_strategies.ResidualAddAdapterStrategy'), _target_='nemo.collections.common.parts.adapter_modules.LinearAdapter')


In [20]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 13.0 M
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 351 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
13.3 M    Trainable params
0         Non-trainable params
13.3 M    Total params
53.297    Total estimated model params size (MB)

In [21]:
model.add_adapter(name=adapter_name, cfg=adapter_cfg)

In [22]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 13.2 M
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 351 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
13.5 M    Trainable params
0         Non-trainable params
13.5 M    Total params
54.041    Total estimated model params size (MB)

In [23]:
model.set_enabled_adapters(enabled=False)  # disable all adapters
model.set_enabled_adapters(name=adapter_name, enabled=True)  # enable only the current adapter we want to train

[NeMo I 2024-07-14 01:07:53 adapter_mixins:719] Setting adapter 'AN4' status : Enabled = False
[NeMo I 2024-07-14 01:07:53 adapter_mixins:734] Setting adapter 'AN4' status : Enabled = True


In [24]:
model.freeze()
model.unfreeze_enabled_adapters()
model.unfreeze()
model.decoder.unfreeze()

[NeMo I 2024-07-14 01:07:53 adapter_mixins:405] Froze module encoder.layers.0.conv.batch_norm: BatchNorm1d(176, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-14 01:07:53 adapter_mixins:405] Froze module encoder.layers.1.conv.batch_norm: BatchNorm1d(176, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-14 01:07:53 adapter_mixins:405] Froze module encoder.layers.2.conv.batch_norm: BatchNorm1d(176, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-14 01:07:53 adapter_mixins:405] Froze module encoder.layers.3.conv.batch_norm: BatchNorm1d(176, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-14 01:07:53 adapter_mixins:405] Froze module encoder.layers.4.conv.batch_norm: BatchNorm1d(176, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-14 01:07:53 adapter_mixins:405] Froze module encoder.layers.5.conv.batch_norm: BatchNorm1d(176, eps

In [25]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 13.2 M
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 351 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
13.5 M    Trainable params
0         Non-trainable params
13.5 M    Total params
54.041    Total estimated model params size (MB)

In [26]:
# Prepare NeMo's Experiment manager to handle checkpoint saving and logging for us
from nemo.utils import exp_manager

# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/',
    name=f"finetune-adpater-decoder-multidomain-4-encoder-unfreeze",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
)

exp_config = OmegaConf.structured(exp_config)

logdir = exp_manager.exp_manager(trainer, exp_config)

[NeMo I 2024-07-14 01:07:53 exp_manager:396] Experiments will be logged at experiments/finetune-adpater-decoder-multidomain-3-encoder-unfreeze/2024-07-14_01-07-53
[NeMo I 2024-07-14 01:07:53 exp_manager:842] TensorboardLogger has been set up


[NeMo W 2024-07-14 01:07:53 exp_manager:952] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to 30000. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


In [27]:
model.cfg['tokenizer']



{'dir': PosixPath('/external2/models/hf/stt_en_conformer_ctc_small/new_tokenizer'), 'type': 'bpe', 'model_path': '/external2/models/hf/stt_en_conformer_ctc_small/new_tokenizer/tokenizer.model', 'vocab_path': '/external2/models/hf/stt_en_conformer_ctc_small/new_tokenizer/vocab.txt', 'spe_tokenizer_vocab': '/external2/models/hf/stt_en_conformer_ctc_small/new_tokenizer/tokenizer.vocab'}

In [28]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 13.2 M
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 351 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
13.5 M    Trainable params
0         Non-trainable params
13.5 M    Total params
54.041    Total estimated model params size (MB)

In [29]:
# Finally, train the adapters
trainer.fit(model)

You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-07-14 01:07:54 modelPT:723] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.1
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2024-07-14 01:07:54 lr_scheduler:915] Scheduler "<nemo.core.optim.lr_scheduler.NoamAnnealing object at 0x778d48286bc0>" 
    will be used during training (effective maximum steps = 30000) - 
    Parameters : 
    (d_model: 176
    warmup_steps: 100
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 30000
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 13.2 M
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 351 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
13.5 M    Trainable params
0         Non-trainable params
13.5 M    Total params
54.041    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo I 2024-07-14 01:07:56 wer:318] 
    
[NeMo I 2024-07-14 01:07:56 wer:319] reference:T787-T849-T718 T714_T718_T767 i T817 T714_T745 am experiencing T817 T714_T765 shortness of breath T817 T714_T818 lately T817 T862_T765_T728
[NeMo I 2024-07-14 01:07:56 wer:320] predicted:le singleT860T634 longkeT538 neT295T352T598T87T737berT295 ourT874T470T538iteT415T519 ne maybeT292T433T884 y heardT538T591T538 fatherT725 hoT519T636 sortT136T821 le hadT295T92T853T224 mhm brother movenkT848T519T270 singleT136 le
[NeMo I 2024-07-14 01:07:56 wer:318] 
    
[NeMo I 2024-07-14 01:07:56 wer:319] reference:T787-T849-T718 T714_T718_T767 i T817 T714_T745 have been feeling T817 T714_T765 extremely tired T817 T714_T897 in the evenings T817 T714_T818 for a few weeks now T817 T862_T765_T728
[NeMo I 2024-07-14 01:07:56 wer:320] predicted:T453T242T860T87 bestT874 ten nT499T519T837T189T538T415T519 had strange had drinkT246T455T304ver currentT526 spT519T455 long scT253T35T710T874atingT295T519ld9culdT245 bri had, m

...

Validation: 0it [00:00, ?it/s]

[NeMo I 2024-07-14 02:46:31 wer:318] 
    
[NeMo I 2024-07-14 02:46:31 wer:319] reference:T787-T849-T718 T714_T718_T767 i T817 T714_T745 am experiencing T817 T714_T765 shortness of breath T817 T714_T818 lately T817 T862_T765_T728
[NeMo I 2024-07-14 02:46:31 wer:320] predicted:domain-iot T3-T317-T58 T1_T448_T176 i T0 aT1_T558 have serinncy T0 T1_ soness b T0 T2_ 
[NeMo I 2024-07-14 02:46:31 wer:318] 
    
[NeMo I 2024-07-14 02:46:31 wer:319] reference:T787-T849-T718 T714_T718_T767 i T817 T714_T745 have been feeling T817 T714_T765 extremely tired T817 T714_T897 in the evenings T817 T714_T818 for a few weeks now T817 T862_T765_T728
[NeMo I 2024-07-14 02:46:31 wer:320] predicted:domain-ot T3-T613-T448 T1_T448_T176 i T0 T1_T558 have beens T0 T1_T389 ated T0 T1_T447 kneen T0 forT1_ my the enee T0 T2_T389_T0
[NeMo I 2024-07-14 02:46:31 wer:318] 
    
[NeMo I 2024-07-14 02:46:31 wer:319] reference:T787-T849-T718 T714_T718_T767 i T817 T714_T745 had T714_T823 a heart attack T817 T714_T775 two ye

Epoch 50, global step 29784: 'val_wer' was not in top 3


[NeMo I 2024-07-14 02:46:46 wer:318] 
    
[NeMo I 2024-07-14 02:46:46 wer:319] reference:domain-iot T3-T613-T448 T1_T448_T176 my son T0 T1_T558 complains of T0 T1_T389 ear pain T0 T1_T447 every night T0 T2_T389_T144
[NeMo I 2024-07-14 02:46:46 wer:320] predicted:domain-iot T3-T613-T448 T1_T448_T176 my son T0 T1_T558 complain of T0 T1_T389 faar pain pain T0 T1_T447 every night T0 T2_T389_T144
[NeMo I 2024-07-14 02:46:55 wer:318] 
    
[NeMo I 2024-07-14 02:46:55 wer:319] reference:domain-iot T3-T118-T58 T1_T558 notify T0 T1_T61 me T0 if there are any changes to my T1_T219_T216 flight T0 to T1_T12 tokyo T0 tomorrow via T1_T282 skyscanner T0 T2_T201_T657
[NeMo I 2024-07-14 02:46:55 wer:320] predicted:domain-iot T3-T118-T58 T1_T558 notify T0 T1_ me account T0 if are changes T0 to my T1_T219_T216 flight T0 toT1_T12 tok T0aorow and T1_T282kysner T0 T2_T523_T219
[NeMo I 2024-07-14 02:47:04 wer:318] 
    
[NeMo I 2024-07-14 02:47:04 wer:319] reference:domain-iot T3-T118-T58 T1_T558 confirm T0

`Trainer.fit` stopped: `max_steps=30000` reached.
Restoring states from the checkpoint path at experiments/finetune-adpater-decoder-multidomain-3-encoder-unfreeze/2024-07-14_01-07-53/checkpoints/finetune-adpater-decoder-multidomain-3-encoder-unfreeze--val_wer=0.9368-epoch=29.ckpt
Restored all states from the checkpoint at experiments/finetune-adpater-decoder-multidomain-3-encoder-unfreeze/2024-07-14_01-07-53/checkpoints/finetune-adpater-decoder-multidomain-3-encoder-unfreeze--val_wer=0.9368-epoch=29.ckpt
